In [1]:
#import requests to make connection to url
import requests
#import beautiful soup to help sort through html code
from bs4 import BeautifulSoup

In [2]:
#create url variable to establish connection
urls = ['https://www.metacritic.com/browse/games/release-date/available/xboxone/metascore?page=',
       'https://www.metacritic.com/browse/games/release-date/available/ps4/metascore?page=',
       'https://www.metacritic.com/browse/games/release-date/available/switch/metascore?page=']

#create header for this particular website as you get a 403 code if it doesnt recognize the system trying to make the connection
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36"}

In [3]:
#scrape all titles and platform they were released on
title = []
platform = []
\

for url in urls:
    
    if url.split('/')[-2] == 'xboxone':
        for page in range(0, 18):
            request = requests.get(url+str(page), headers=headers)
            soup = BeautifulSoup(request.content, 'html.parser')
            
            titles = [t.string for t in soup.findAll('a', class_=['title'])]
            pltfrm = [p.find('span', class_='data').string.strip() for p in soup.findAll('div', class_=['clamp-details'])]

            title.extend(titles)
            platform.extend(pltfrm)
            
    elif url.split('/')[-2] == 'ps4':
        for page in range(0, 25):
            request = requests.get(url+str(page), headers=headers)
            soup = BeautifulSoup(request.content, 'html.parser')
            
            titles = [t.string for t in soup.findAll('a', class_=['title'])]
            pltfrm = [p.find('span', class_='data').string.strip() for p in soup.findAll('div', class_=['clamp-details'])]

            title.extend(titles)
            platform.extend(pltfrm)
            
    elif url.split('/')[-2] == 'switch':
        for page in range(0, 18):
            request = requests.get(url+str(page), headers=headers)
            soup = BeautifulSoup(request.content, 'html.parser')
            
            titles = [t.string for t in soup.findAll('a', class_=['title'])]
            pltfrm = [p.find('span', class_='data').string.strip() for p in soup.findAll('div', class_=['clamp-details'])]
            
            title.extend(titles)
            platform.extend(pltfrm)

In [4]:
#combine platform and title into a subset of the url that will be used to collect user reviews
url_title = []
for i in range(len(title)):
    t_for_url = "-".join(title[i].lower().replace(":","").replace("'","").split())
    p_for_url = "-".join(platform[i].lower().split())
    url_title.append(p_for_url + '/' + t_for_url)

In [5]:
import numpy as np
import re

url_base = 'https://www.metacritic.com/game/'
sentiment_pattern = r'\b(positive|mixed|negative)\b'
num_of_reviews = 15

game_title = []
console = []
metascore = []
metasentiment = []
avg_userscore = []
avg_usersentiment = []
developer = []              
genre = []                  
players = []
esrb_rating = []
release_date = []
username = []
userscore = []
usersentiment = []
review = []
review_date = []

for i in range(len(url_title)):
    response = requests.get(url_base + url_title[i], headers = headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    #none type has no attribute find, try except for commented out code will be filled with np.nan to represent null
    ttl = title[i]
    pltfrm = platform[i]
    
    try:
        mscore = soup.find('a', class_=['metascore_anchor'], href=["/game/" + url_title[i] + "/critic-reviews"]).find('span').get_text()
    except AttributeError:
        mscore = np.nan
        
    try:
        msentiment = soup.find('a', class_=['metascore_anchor'], href=["/game/" + url_title[i] + "/critic-reviews"]).find('div')['class'][-1]
    except AttributeError:
        msentiment = np.nan
        
    try:
        avg_uscore = soup.find('a', class_=['metascore_anchor'], href=["/game/" + url_title[i] + "/user-reviews"]).find('div').get_text()
    except AttributeError:
        avg_uscore = np.nan
        
    try:
        avg_usentiment = soup.find('a', class_=['metascore_anchor'], href=["/game/" + url_title[i] + "/user-reviews"]).find('div')['class'][-1]
    except AttributeError:
        avg_usentiment = np.nan
    
    try:
        dvlpr = soup.find('li', class_=['summary_detail developer']).find('span', class_=['data']).get_text(strip=True)
    except AttributeError:
        dvlpr = np.nan
    
    try:
        gnr = [genre.get_text(strip=True) for genre in soup.find('li', class_=['summary_detail product_genre']).findAll('span', class_=['data'])]
    except AttributeError:
        gnr = np.nan
        
    try:
        num_of_players = soup.find('li', class_=['summary_detail product_players']).find('span', class_=['data']).get_text(strip=True)
    except AttributeError:
        num_of_players = np.nan
    
    try:
        esrb = soup.find('li', class_=['summary_detail product_rating']).find('span', class_=['data']).get_text()
    except AttributeError:
        esrb = np.nan
        
    try:
        rel_date = soup.find('li', class_=['summary_detail release_data']).find('span', class_=['data']).get_text()
    except AttributeError:
        rel_date = np.nan
    
    
    response2 = requests.get(url_base + url_title[i] + '/user-reviews', headers = headers)
    soup2 = BeautifulSoup(response2.content, 'html.parser')
    
    reviews = soup2.findAll('li', class_=['user_review'])

    if len(reviews) >= num_of_reviews:
        for rev in reviews[:num_of_reviews]:
            uname = rev.find('a').get_text()
            uscore = rev.find('div', class_=['review_grade']).div.get_text()
            match = re.search(sentiment_pattern, ' '.join(rev.find('div', class_=['review_grade']).div['class']))
            usentiment = match.group()
            rvw = rev.find('div', class_=['review_body']).get_text(strip=True)
            rev_date = rev.find('div', class_=['date']).get_text()
            
            game_title.append(ttl)
            console.append(pltfrm)
            metascore.append(mscore)
            metasentiment.append(msentiment)
            avg_userscore.append(avg_uscore)
            avg_usersentiment.append(avg_usentiment)
            developer.append(dvlpr)
            genre.append(gnr)
            players.append(num_of_players)
            esrb_rating.append(esrb)
            release_date.append(rel_date)
            username.append(uname)
            userscore.append(uscore)
            usersentiment.append(usentiment)
            review.append(rvw)
            review_date.append(rev_date)

In [6]:
import pandas as pd

scraped = {'title': game_title,
           'platform': console,
           'metascore': metascore,
           'metasentiment': metasentiment,
           'average_userscore': avg_userscore,
           'average_usersentiment': avg_usersentiment,
           'developer': developer,
           'genre': genre,
           'number_of_players': players,
           'esrb_rating': esrb_rating,
           'release_date':release_date,
           'username': username,
           'userscore': userscore,
           'usersentiment': usersentiment,
           'review': review,
           'reveiw_date': review_date
          }

df = pd.DataFrame(scraped)

pd.set_option('display.max_columns', None)
df.head()

,title,platform,metascore,metasentiment,average_userscore,average_usersentiment,developer,genre,number_of_players,esrb_rating,release_date,username,userscore,review,reveiw_date
0,Red Dead Redemption 2,Xbox One,97,positive,7.9,positive,Rockstar Games,"[Action Adventure, Open-World]",Up to 32,M,"Oct 26, 2018",gnadenlos,7,"The main problem is, that it's not a real open...","Nov 1, 2018"
1,Red Dead Redemption 2,Xbox One,97,positive,7.9,positive,Rockstar Games,"[Action Adventure, Open-World]",Up to 32,M,"Oct 26, 2018",Feriatus,7,It's not a bad game but the gameplay is an out...,"Oct 29, 2018"
2,Red Dead Redemption 2,Xbox One,97,positive,7.9,positive,Rockstar Games,"[Action Adventure, Open-World]",Up to 32,M,"Oct 26, 2018",ponux,7,"Visually superb (except cutscenes), good (not ...","Nov 5, 2018"
3,Red Dead Redemption 2,Xbox One,97,positive,7.9,positive,Rockstar Games,"[Action Adventure, Open-World]",Up to 32,M,"Oct 26, 2018",Picklock,5,"Great looking game backed up by clumsy, overly...","Nov 4, 2018"
4,Red Dead Redemption 2,Xbox One,97,positive,7.9,positive,Rockstar Games,"[Action Adventure, Open-World]",Up to 32,M,"Oct 26, 2018",Saints,6,Red Dead Redemption 2 is an amazing game that ...,"Oct 30, 2018"


In [7]:
df.tail()

,title,platform,metascore,metasentiment,average_userscore,average_usersentiment,developer,genre,number_of_players,esrb_rating,release_date,username,userscore,review,reveiw_date
20770,Vroom in the Night Sky,Switch,17,negative,3.2,negative,Poisoft,"[Sports, Individual, Biking]",No Online Multiplayer,E,"Apr 5, 2017",toonlinkman,0,This game is an absolute disgrace. You should ...,"Apr 29, 2017"
20771,Vroom in the Night Sky,Switch,17,negative,3.2,negative,Poisoft,"[Sports, Individual, Biking]",No Online Multiplayer,E,"Apr 5, 2017",AllISayIsRight,0,"Best game ever. It has great music, great grap...","Dec 18, 2017"
20772,Vroom in the Night Sky,Switch,17,negative,3.2,negative,Poisoft,"[Sports, Individual, Biking]",No Online Multiplayer,E,"Apr 5, 2017",MA3GNetwork,10,"This is the best Switch game in existance, I h...","Oct 27, 2017"
20773,Vroom in the Night Sky,Switch,17,negative,3.2,negative,Poisoft,"[Sports, Individual, Biking]",No Online Multiplayer,E,"Apr 5, 2017",wariowario,9,I saw footage of the Japanese version on the e...,"Apr 7, 2017"
20774,Vroom in the Night Sky,Switch,17,negative,3.2,negative,Poisoft,"[Sports, Individual, Biking]",No Online Multiplayer,E,"Apr 5, 2017",thedonald112915,10,On switch there are many great games such as b...,"Dec 1, 2017"


In [8]:
df.to_csv('MetacriticGameReviews.csv')